In [ ]:
from keras.models import Model, load_model
from keras.layers import Input,  Dense, GlobalAveragePooling2D, BatchNormalization, Lambda
from keras.layers import Dropout, Conv2D, PReLU, MaxPooling2D, Reshape, Flatten, Activation
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
train_dir = '/datasets/hand/gesture/mini_gesture/train'
val_dir = '/datasets/hand/gesture/mini_gesture/val'
num_label = 7
batch_size = 256
num_of_epochs = 100
im_width, im_height = 200, 200

In [ ]:
# Create model
nn_input = Input(shape=(im_width, im_height, 1))
normalize = lambda x: x / 255
x = Lambda(normalize)(nn_input)
x = Conv2D(32, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Flatten()(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_label)(x)
predictions = Activation('softmax')(x)

model = Model(inputs=nn_input, outputs=predictions)
# We add metrics to get more results you want to see
model.compile(optimizer=Adam(lr=0.0001, decay=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Data Augmentation

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

val_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(im_width, im_height),
    class_mode="categorical",
    color_mode="grayscale",
    batch_size=batch_size,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(im_width, im_height),
    class_mode="categorical",
    color_mode="grayscale",
    batch_size=batch_size,
)

# mapping of label name and class index 
train_generator.class_indices

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/log/tensorboard', batch_size=batch_size)

# Save best model
save_model = ModelCheckpoint("weights.{epoch:02d}-{acc:.2f}-{loss:.4f}.hdf5",
                                 monitor="loss",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="auto")

In [ ]:
# Fit the model on the batches generated by datagen.flow().
# model.load_weights('weights.49-0.72-0.7661.hdf5')
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    validation_data=val_generator,
                    callbacks = [tensor_board, save_model])

# Save the best model
# model = load_model('model.49-0.72-0.7661.hdf5')
# model.save('gesture_model.hdf5')

In [ ]:
sys.path.insert(0,'../tool')
from keras_to_tensorflow import save_frozen_tf_model
save_frozen_tf_model(model, 'gesture_model.pb')